In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

!git clone https://github.com/zzilch/RPLAN-Toolbox



Mounted at /content/drive/
Cloning into 'RPLAN-Toolbox'...
remote: Enumerating objects: 55, done.
remote: Counting objects: 100% (55/55), done.
remote: Compressing objects: 100% (41/41), done.
remote: Total 55 (delta 16), reused 45 (delta 11), pack-reused 0
Receiving objects: 100% (55/55), 53.36 KiB | 3.81 MiB/s, done.
Resolving deltas: 100% (16/16), done.


# Modifiche

Nel file floorplan.py, presente nella cartella rplna, bisonga modificare il codice alla riga 208, eliminando un [0], il codice diventa:
    
    c = stats.mode(self.category[region.coords[:,0],region.coords[:,1]])[0]


# Modifiche
Per omologare il formato to_dict a quello utilizzato in iPLAN, si deve modificare il file da

    def to_dict(self,xyxy=True,dtype=int):
          '''
          Compress data, notice:
          !!! int->uint8: a(uint8)+b(uint8) may overflow !!!
          '''
          return {
              'name'      :self.name,
              'types'     :self.rooms[:,-1].astype(dtype),
              'boxes'     :(self.rooms[:,[1,0,3,2]]).astype(dtype)
              if xyxy else self.rooms[:,:4].astype(dtype),
              'boundary'  :self.exterior_boundary[:,[1,0,2,3]].astype(dtype)
              if xyxy else self.exterior_boundary.astype(dtype),
              'edges'     :self.edges.astype(dtype)
          }

a

    def to_dict(self,xyxy=True,dtype=int):
        '''
        Compress data, notice:
        !!! int->uint8: a(uint8)+b(uint8) may overflow !!!
        '''
        return {
            'name': self.name,
            'gt_rTypes': self.rooms[:,-1].astype(dtype),
            'gt_rBoxes': (self.rooms[:,[1,0,3,2]]).astype(dtype)
            if xyxy else self.rooms[:,:4].astype(dtype),
            'Boundary': self.exterior_boundary[:,[1,0,2,3]].astype(dtype)
            if xyxy else self.exterior_boundary.astype(dtype),
            'rTypes' : np.array([]).astype(dtype),
            'rBoxes' : np.array([]).astype(dtype),
            'rCenters' : np.array([]).astype(dtype)
        }


Per ovviare al problema che il modello iPLAN prende solo immagini 128×128 la funzione veine modificata ulteriormente in:

    def to_dict(self, xyxy=True, dtype=int):
        '''
        Compress data, notice:
        !!! int->uint8: a(uint8)+b(uint8) may overflow !!!
        '''
        return {
            'name': self.name,
            'gt_rTypes': self.rooms[:, -1].astype(dtype),
            'gt_rBoxes': ((self.rooms[:, [1, 0, 3, 2]]).astype(dtype) / 255) * 127
                          if xyxy else (self.rooms[:, :4].astype(dtype) / 255) * 127,
            'Boundary': (self.exterior_boundary[:, [1, 0, 2, 3]].astype(dtype) / 255) * 127
                          if xyxy else (self.exterior_boundary.astype(dtype) / 255) * 127,
            'rTypes': np.array([]).astype(dtype),
            'rBoxes': np.array([]).astype(dtype),
            'rCenters': np.array([]).astype(dtype)
        }


In [ ]:
import os
print("Directory di lavoro corrente:", os.getcwd())
RPLAN_directory = '/content/RPLAN-Toolbox/'
os.chdir(RPLAN_directory)
print("Nuova directory di lavoro:", os.getcwd())
from rplan.floorplan import Floorplan

Directory di lavoro corrente: /content
Nuova directory di lavoro: /content/RPLAN-Toolbox


In [ ]:
from pathlib import Path
import scipy.io as sio

In [ ]:
RPLAN_DIR = Path("/content/drive/MyDrive/Colab_Notebooks/Deep_Learning/Progetto/Input/Input_resize")
for mat in RPLAN_DIR.iterdir():
  number = str(mat).split('/')[-1].split('.')[0]
  fp = Floorplan(str(mat))
  data = fp.to_dict()
  sio.savemat(f"/content/drive/MyDrive/Colab_Notebooks/Deep_Learning/Progetto/Data/{number}.mat", {"data" : data})


In [ ]:
data = fp.to_dict()
data

{'name': '2',
 'gt_rTypes': array([3, 2, 7, 1, 8, 0, 9]),
 'gt_rBoxes': array([[ 81.18039216,  11.45490196,  94.62745098,  30.38039216],
        [ 33.36862745,  11.45490196,  48.80784314,  38.84705882],
        [ 59.26666667,  82.17647059,  86.16078431, 105.08627451],
        [ 33.36862745,  73.21176471,  56.77647059, 105.08627451],
        [ 33.36862745,  41.3372549 ,  56.77647059,  70.72156863],
        [ 51.29803922,  11.45490196,  86.16078431,  79.68627451],
        [ 59.26666667, 107.57647059,  86.16078431, 116.54117647]]),
 'Boundary': array([[ 86.16078431,  32.37254902,   0.49803922,   0.49803922],
        [ 86.16078431,  40.83921569,   0.49803922,   0.49803922],
        [ 86.16078431, 116.54117647,   0.99607843,   0.        ],
        [ 59.26666667, 116.54117647,   1.49411765,   0.        ],
        [ 59.26666667, 105.08627451,   0.99607843,   0.        ],
        [ 33.36862745, 105.08627451,   1.49411765,   0.        ],
        [ 33.36862745,  11.45490196,   0.        ,   0.  

In [ ]:
import numpy as np

In [ ]:
def getList(path):
  mat_list = os.listdir(path)
  mat_list.sort()
  temp_list = []
  for name in mat_list[:]:
      mat_path = os.path.join(path, name)
      data = sio.loadmat(mat_path, squeeze_me=True, struct_as_record=False)['data']
      if type(data.rTypes) is not np.ndarray or len(data.rTypes) == 0:
          temp_list.append(name)

  return temp_list

In [ ]:
data_root = "/content/drive/MyDrive/Colab_Notebooks/Deep_Learning/Progetto/Data"
floorplans = getList(data_root)

In [ ]:
floorplans

['0.mat',
 '1.mat',
 '10.mat',
 '2.mat',
 '3.mat',
 '4.mat',
 '5.mat',
 '6.mat',
 '7.mat',
 '8.mat',
 '9.mat']